# Nuclia Understanding | Nuclia Understanding

> [Nuclia](https://nuclia.com)は、内部および外部のあらゆるソースからの非構造化データを自動的にインデックス化し、最適化された検索結果と生成的な回答を提供します。ビデオとオーディオの書き起こし、画像コンテンツの抽出、文書のパースが可能です。
>
> > [Nuclia](https://nuclia.com) automatically indexes your unstructured data from any internal and external source, providing optimized search results and generative answers. It can handle video and audio transcription, image content extraction, and document parsing.

`Nuclia Understanding API`は、テキスト、ウェブページ、ドキュメント、オーディオ/ビデオコンテンツを含む非構造化データの処理をサポートしています。必要に応じて音声認識やOCRを使用して、どこにでもあるテキストを抽出し、エンティティを識別し、メタデータやPDF内の画像のような埋め込まれたファイル、ウェブリンクも抽出します。さらに、コンテンツの要約を提供します。

> The `Nuclia Understanding API` supports the processing of unstructured data, including text, web pages, documents, and audio/video contents. It extracts all texts wherever it is (using speech-to-text or OCR when needed), it identifies entities, it aslo extracts metadata, embedded files (like images in a PDF), and web links. It also provides a summary of the content.

`Nuclia Understanding API`を使用するためには、`Nuclia`アカウントが必要です。<https://nuclia.cloud>で無料でアカウントを作成し、その後[NUAキーを作成する](https://docs.nuclia.dev/docs/docs/using/understanding/intro)手順はこちらです。

> To use the `Nuclia Understanding API`, you need to have a `Nuclia` account. You can create one for free at <https://nuclia.cloud>, and then [create a NUA key](https://docs.nuclia.dev/docs/docs/using/understanding/intro).




In [ ]:
#!pip install --upgrade protobuf
#!pip install nucliadb-protos

In [ ]:
import os

os.environ["NUCLIA_ZONE"] = "<YOUR_ZONE>"  # e.g. europe-1
os.environ["NUCLIA_NUA_KEY"] = "<YOUR_API_KEY>"

In [ ]:
from langchain.tools.nuclia import NucliaUnderstandingAPI

nua = NucliaUnderstandingAPI(enable_ml=False)

Nuclia Understanding APIにファイルをプッシュするには、`push`アクションを使用します。処理は非同期で行われるため、結果はファイルがプッシュされた順番とは異なる順序で返される可能性があります。そのため、結果を対応するファイルと照合するためには`id`を提供する必要があります。

> You can push files to the Nuclia Understanding API using the `push` action. As the processing is done asynchronously, the results might be returned in a different order than the files were pushed. That is why you need to provide an `id` to match the results with the corresponding file.




In [ ]:
nua.run({"action": "push", "id": "1", "path": "./report.docx"})
nua.run({"action": "push", "id": "2", "path": "./interview.mp4"})

JSON形式の結果が得られるまで、`pull`アクションをループ内で呼び出すことができるようになりました。

> You can now call the `pull` action in a loop until you get the JSON-formatted result.




In [ ]:
import time

pending = True
data = None
while pending:
    time.sleep(15)
    data = nua.run({"action": "pull", "id": "1", "path": None})
    if data:
        print(data)
        pending = False
    else:
        print("waiting...")

`async`モードでは一つのステップで実行することもできます。プッシュを行うだけで、結果がプルされるまで待機します。

> You can also do it in one step in `async` mode, you only need to do a push, and it will wait until the results are pulled:




In [ ]:
import asyncio


async def process():
    data = await nua.arun(
        {"action": "push", "id": "1", "path": "./talk.mp4", "text": None}
    )
    print(data)


asyncio.run(process())

## Retrieved information | 取得した情報

Nucliaは以下の情報を返します：

> Nuclia returns the following information:

* ファイルのメタデータ

  > file metadata

* 抽出されたテキスト

  > extracted text

* 埋め込まれた画像内のテキストのような、ネストされたテキスト

  > nested text (like text in an embedded image)

* `enable_ml`が`True`に設定されている場合にのみ、要約

  > a summary (only when `enable_ml` is set to `True`)

* 段落と文の分割（それぞれの最初と最後の文字の位置によって定義され、ビデオやオーディオファイルの場合は開始時刻と終了時刻によっても定義される）

  > paragraphs and sentences splitting (defined by the position of their first and last characters, plus start time and end time for a video or audio file)

* 固有名詞：人物、日付、場所、組織など（`enable_ml`が`True`に設定されている場合のみ）

  > named entities: people, dates, places, organizations, etc. (only when `enable_ml` is set to `True`)

* リンク

  > links

* サムネイル

  > a thumbnail

* 埋め込まれたファイル

  > embedded files

* テキストのベクトル表現（`enable_ml`が`True`に設定されている場合のみ）

  > the vector representations of the text (only when `enable_ml` is set to `True`)


注意：

> Note:

生成されたファイル（サムネイル、抽出された埋め込みファイルなど）はトークンとして提供されます。これらは[`/processing/download` エンドポイント](https://docs.nuclia.dev/docs/api#operation/Download_binary_file_processing_download_get)を使用してダウンロードできます。

> Generated files (thumbnail, extracted embedded files, etc.) are provided as a token. You can download them with the [`/processing/download` endpoint](https://docs.nuclia.dev/docs/api#operation/Download_binary_file_processing_download_get).

また、任意のレベルで属性が特定のサイズを超えた場合、それはダウンロード可能なファイルに格納され、ドキュメント内では `{"file": {"uri": "JWT_TOKEN"}}` の形式のファイルポインターに置き換えられます。ルールとして、メッセージのサイズが100万文字を超えると、最も大きな部分がダウンロード可能なファイルに移動されます。最初に、圧縮プロセスはベクターを対象にします。それが不十分な場合、次に大規模なフィールドメタデータを対象にし、最終的には抽出されたテキストを対象にします。

> Also at any level, if an attribute exceeds a certain size, it will be put in a downloadable file and will be replaced in the document by a file pointer. This will consist of `{"file": {"uri": "JWT_TOKEN"}}`. The rule is that if the size of the message is greater than 1000000 characters, the biggest parts will be moved to downloadable files. First, the compression process will target vectors. If that is not enough, it will target large field metadata, and finally it will target extracted text.


